# Discussing **Cooperative Cancellation**

- **Structured concurrency** is discussed.
- **Cancelling** jobs and **timeout** cancellation are briefly covered.

## **Structured Concurrency** and **Cooperative Cancellation**

1. **Coroutines Must Have a Well-Defined Scope**: All coroutines must be launched within a specific **scope** (e.g., `CoroutineScope`, `runBlocking`, or a lifecycle scope). When this scope is **cancelled**, *all coroutines launched within it are automatically cancelled.*
2. **Cancellation Propagation**: If the *parent scope* is cancelled, all the coroutines within that scope are cancelled as well. This helps *prevent memory leaks* by ensuring that no coroutines are left running without control (There is an exception that will be discussed).
3. **Hierarchy of Coroutines**: Child coroutines are bound to their parent coroutines. If a parent coroutine is cancelled or fails, *all its child coroutines are cancelled as well*. This creates a structured tree of coroutines, where failures and cancellations are propagated correctly.
4. **Exceptions Handling**: In structured concurrency, *exceptions are propagated through the coroutine hierarchy*. If a child coroutine throws an exception, it affects the parent coroutine unless caught.

**NOTE**: When cancellation request is sent by calling `cancel()`, it is JUST initiated; The point at which the parent coroutine and its children cancel, **depends on the structure of the code**.

In [1]:
// Importing some neccessary libs in jupyter notebook
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.2")
import kotlinx.coroutines.*

In [2]:
// Cancelling a job

/*
'cancel()' command is managed by kotlin. 
It activates on suspension points or when a coroutine scope reaches to the end of its blaock {}.
*/
fun main() = runBlocking { 
    launch {
        print("Hello ")
        cancel()  // Cancels the coroutine when it gets suspended or scope reached to the end.
        delay(1_000) // If commented, next line will be shown.
        print("Word")  // Will not be shown
    } 
}

main()

Hello 

StandaloneCoroutine{Cancelled}@329c3122

In [3]:
// Cancelling a job without suspend point

/*
'cancel()' command is managed by kotlin. It activates when a coroutine suspends or its scope finishes.
*/
fun main() = runBlocking { 
    launch {
        print("Hello ")
        cancel()  // Cancels the coroutine and its children when it gets to the end of the scope.
        print("Word")  // Will be shown

        // Will not get executed
        launch {
            println("Inner coroutine") 
        }
    } // <-- Cancellation activates here; end of the coroutine scope
}

main()

Hello Word

StandaloneCoroutine{Cancelled}@1b12d691

In [4]:
// Cancellation when reaching to a suspend function

fun main() = runBlocking { 
    launch {
        print("Hello ")
        cancel()  
        print("Word \n")  // Will be shown

        susFun1()  // Will get executed because this function never gets suspended.
        susFun2()  // Cancellation happens here because there is a suspention point in the function.
    } 
}

suspend fun susFun1() {
    println("susFun1 got executed") 
}

suspend fun susFun2() {
    delay(1000) // Suspends function and cancellation happens.
    println("susFun2 got executed")  // Won't be shown.
}

main()

Hello Word 
susFun1 got executed


StandaloneCoroutine{Cancelled}@5f6a2269

## Cancellation and Custom **Corotoutine Scope**

 - `coroutineScope` in another scope, inherit its job and cancellation applies when:
    1. IF the `coroutineScope` **suspends** in the parent scope:
        > Cancellation happens at the point of suspension. It is actually just a suspend function with a coroutine scope.
    4. IF the `coroutineScope` has **no suspension point**:    
       > Cancellation happens **AT THE END OF** the scope.
       Because the coroutine scope has inheritted outer scope's job that is already received
       cancellation request. 
 - A custom `CorotuineScope` can act independently based on its configurations.

In [5]:
// Cancellation when reaching to a child 'coroutineScope' with inheretted 'Job'

fun main() = runBlocking { 
    launch {
        println("Program Started.")
        print("Hello ")
        cancel()  // Cancels the coroutine and its children when it gets to the end of the scope.
        print("Word \n")  // Will be shown

        /* 
        IF 'corScopeFun' suspends:
            cancellation happens at the point of suspension like before.
            
        IF 'corScopeFun' has no suspension point:    
            Cancellation happens at the end of corScopeFun's scope '{}'.
        */
        corScopeFun()  
        println("Program finished.") // Won't be shown.
    }
}

suspend fun corScopeFun() = coroutineScope {
    // delay(1000)  // Uncomment to see the difference.
    println("corScopeFun got executed") 
    launch {
        println("corScopeFun launched a coroutine") // Won't be shown
    }
} // Will cancels here before any launches if there was no suspension point.

main()

Program Started.
Hello Word 
corScopeFun got executed


StandaloneCoroutine{Cancelled}@3303f449

In [6]:
// Cancellation when reaching to a custom 'coroutineScope' with its own 'Job'

// Creating a custom context
val myScopeJob = Job();
val context = Dispatchers.IO + myScopeJob + CoroutineName("MyContext")
val myScope = CoroutineScope(context)

fun main() = runBlocking { 
    
    val outerJob = myScope.launch {
        delay(3000) 
        println("myScope first coroutine got executed") 
    }    
    
    var innerJob: Job? = null 
    
    launch mainJob@{
        println("Program Started.")
        print("Hello ")
        cancel()  
        print("Word \n")  // Will be shown

        // delay(1000)  // Uncomment to see the difference; 'launch' gets cancelled here and 'innerJob' never gets initiated.
        
        /*
        'myScope' has its own context and job, so 'cancel()' command above won't apply on it.
        'innerJob' is tied to 'myScopeJob' of 'myScope' not the coroutine it is launched in (mainJob).
        */
        innerJob = myScope.launch {
            delay(1000) 
            println("myScope second coroutine got executed") 
        }
        delay(1000)
        println("Program finished.") // Won't be shown.
    }.join()

    innerJob?.join()
    outerJob.join()
    
    myScope.cancel()
}

main()

Program Started.
Hello Word 
myScope second coroutine got executed
myScope first coroutine got executed


## Cancellation Request from a `coroutineScope` and `JobCancellationException`

- When a cancellation request is sent, a `JobCancellationException` will be thrown at the **cancelation point** that most of the time Kotlin catches (as its normal behaviour) and cancels related jobs.
- We can use `try-catch-finaly` to close any resources or prevent cancellation at that specific point; the cancelation will get **postponed to the next point** of suspension or the end of the scope.
- Cancellation is not **Immediate** but it is **Cooperative**.
- `coroutineScope`s builder **inherit its job**, so cancellation request inside a coroutine scope, will cancel its inherited job.
- So if a `coroutineScope` builder is used directly inside a `runBlocking`, it will lead to **uncaught** `JobCancellationException` error because the program's root job has been cancelled.

In [7]:
fun main() = runBlocking { 
    
    println("Program Started.")

    /*
    This coroutine scope inherits 'runBlocking' context and job so cancelling request
    on it, cancels 'runBlocking' job. This results in an Uncaught run-time exception.
    */
    coroutineScope {
        cancel() // This will throw uncaught cancellation Exception on runBlocking.
        println("Scope started.")
        launch {
            println("Hello World from launch") // Won't be shown
        }
        println("Scope finished.")
    } // <-- cancels at this point; cancellation is applied to 'runBlocking' context.

    // Exception breaks the code execution.
    println("Program finished.") // Won't print
}

main()

Program Started.
Scope started.
Scope finished.


ScopeCoroutine was cancelled
kotlinx.coroutines.JobCancellationException: ScopeCoroutine was cancelled; job=ScopeCoroutine{Cancelled}@640b79e0


In [8]:
// Catching a cancellation request

fun main() = runBlocking { 
    
    println("Program Started.")
    
    launch {
        cancel() 
        println("Scope started.")
        launch {
            println("Hello World from launch") // Won't be shown
        }

        try {
            delay(1000) // Cancellation won't happen here because the error has been catched.
        } catch (e: Exception) {
            // Error can be re-thrown if neccessary.
        }
        println("Scope finished.")
    } 

    println("Program finished.") 
}

main()

Program Started.
Program finished.
Scope started.
Scope finished.


In [9]:
// 'coroutineScope' inherits its outer coroutine job and cancels it.

fun main() = runBlocking { 
    
    println("Program Started.")
    
    launch {
        coroutineScope { // Replace with 'launch' to see what will happen.
            cancel()
        }
        println("Hello World.") // Won't be shown
    } 

    println("Program finished.") 
}

main()

Program Started.
Program finished.


## Calling `cancel()` in `suspend` Functions

In [10]:
// 'suspend' functions inherits its outer coroutine job and cancels it.

fun main() = runBlocking { 

    println("Program Started.")
    
    launch {
        // We need to define a nested function to get access to the parent ('runBlocking') context
        suspend fun somSus() {
            cancel()
        }
        somSus()
        delay(1000)
        println("Hello World.") // Won't be shown
    } 

    println("Program finished.") 
}

main()

Program Started.
Program finished.


## Using `withContext(NonCancellable) {...}`

In [11]:
fun main() = runBlocking { 

    println("Program Started.")
    
    launch {
        cancel()
        
        // Canellation request just ignores this
        withContext(NonCancellable) {
            println("Non-cancellable was executed.")
        }
        
        delay(1000)  // Cancelation request applies here 
        println("Hello World.") // Won't be shown
    } 

    println("Program finished.") 
}

main()

Program Started.
Program finished.
Non-cancellable was executed.


## Using `withTimeout` and `withTimeoutOrNull`

- `withTimeout` throws `TimeoutCancellationException` (which is a subclass of `CancellationException`) when the given time interval is passed.
- `withTimeoutOrNull` returns `null` on timeout instead of throwing exceptions.

In [12]:
// Using 'withTimeout'

fun main() = runBlocking { 

    println("Program Started.")
    
    launch root@{
        println("withTimeout was launched.")
        
        // Will cancel 'root' corotoutine job on timeout.
        val res = withTimeout(3_000) {
            println("withTimeout started.")
            println("wait...")
            delay(4_000) // a delay longer than timeout 
            println("withTimeout finished.")
            return@withTimeout 1
        }
        println("corotuine finished: $res.") // Won't be shown on timeout.
    }.join()

    println("Program finished.") 
}

main()

Program Started.
withTimeout was launched.
withTimeout started.
wait...
Program finished.


In [13]:
// Using 'withTimeoutOrNull'

fun main() = runBlocking { 

    println("Program Started.")
    
    launch root@{
        println("withTimeout was launched.")

        // WON'T cancel 'root' corotoutine job on timeout but returns Null.
        val res = withTimeoutOrNull(3_000) {
            println("withTimeout started.")
            println("wait...")
            delay(4_000) // a delay longer than timeout 
            println("withTimeout finished.")
            return@withTimeoutOrNull 1
        }
        println("corotuine finished: $res.") // Will be shown and on timeout res == Null.
    }.join()

    println("Program finished.") 
}

main()

Program Started.
withTimeout was launched.
withTimeout started.
wait...
corotuine finished: null.
Program finished.


## **DYE**: Do Your Experiment

In [14]:
// Ready for your codes